In [1]:
from settings import *

import os
os.environ["CUDA_VISIBLE_DEVICES"] = ",".join(map(str, CUDA_DEVICES))

LOG_FILE = "train_log7.txt"

In [2]:
import sys

from models import attgan
from settings import *
from tensorflow.keras import losses, optimizers, metrics

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import dataloader
import utils

In [3]:
tf.test.is_gpu_available()

True

In [4]:
strategy = tf.distribute.MirroredStrategy(devices=[
    f"/gpu:{idx}" for idx in CUDA_DEVICES
])

In [5]:
train_dloader = dataloader.DataLoader("train", BATCH_SIZE)
valid_dloader = dataloader.DataLoader("valid", BATCH_SIZE)
test_dloader = dataloader.DataLoader("test", BATCH_SIZE)

with strategy.scope():
    train_dloader = tf.data.Dataset.from_generator(
        train_dloader.next_batch, 
        (tf.float32, tf.float32), 
        (
            tf.TensorShape([None, HEIGHT, WIDTH, CHANNEL]), 
            tf.TensorShape([None, NUM_ATT])
        )
    )
    train_dloader = strategy.experimental_distribute_dataset(train_dloader)

    valid_dloader = tf.data.Dataset.from_generator(
        valid_dloader.next_batch, 
        (tf.float32, tf.float32), 
        (
            tf.TensorShape([None, HEIGHT, WIDTH, CHANNEL]), 
            tf.TensorShape([None, NUM_ATT])
        )
    )
    valid_dloader = strategy.experimental_distribute_dataset(valid_dloader)

    test_dloader = tf.data.Dataset.from_generator(
        test_dloader.next_batch, 
        (tf.float32, tf.float32), 
        (
            tf.TensorShape([None, HEIGHT, WIDTH, CHANNEL]), 
            tf.TensorShape([None, NUM_ATT])
        )
    )
    test_dloader = strategy.experimental_distribute_dataset(test_dloader)

In [6]:
with strategy.scope():
    model = attgan.AttGAN()
    
    criterion_MAE = losses.MeanAbsoluteError(reduction=tf.losses.Reduction.SUM)
    criterion_BCE = losses.BinaryCrossentropy(reduction=tf.losses.Reduction.SUM)
    
    optimizer = optimizers.Adam(learning_rate=ETA)

    loss_mean_gen = metrics.Mean()
    loss_mean_dis = metrics.Mean()

In [7]:
# def criterion_MAE(y_true, y_pred):
#     n = y_true.shape[0]

#     loss = tf.math.abs(y_true - y_pred)
#     loss = tf.reshape(loss, shape=(n, -1))
#     # loss = tf.reduce_sum(loss, axis=-1)
#     loss = tf.reduce_mean(loss)

#     return loss

# def criterion_BCE(y_true, y_pred):
#     n = y_true.shape[0]

#     loss = - y_true * tf.math.log(y_pred + 1e-6) - (1 - y_true) * tf.math.log(1 - y_pred + 1e-6)
#     loss = tf.reshape(loss, (n, -1))
#     # loss = tf.reduce_sum(loss, axis=1)
#     loss = tf.reduce_mean(loss)

#     return loss

In [8]:
with strategy.scope():
    def generator_loss(x, a, b, x_rec_a, x_rec_b, d_x, d_rec_a, d_rec_b, c_x, c_rec_a, c_rec_b, training=False):
        n = x.shape[0]

        loss_rec = criterion_MAE(x, x_rec_a)
        loss_adv_gen = tf.reduce_mean(-d_rec_b, axis=0)
        loss_att_gen = criterion_BCE(a, c_rec_a) + criterion_BCE(b, c_rec_b)

        loss_gen = loss_rec*100 + loss_adv_gen + loss_att_gen

        # print("GENERATOR")
        # print("LOSS_GEN:", loss_gen)
        # print("REC:", loss_rec)
        # print("ADV:", loss_adv_gen)
        # print("ATT:", loss_att_gen)
        # print()

        return loss_gen

In [9]:
with strategy.scope():
    def discriminator_loss(x, a, b, x_rec_a, x_rec_b, d_x, d_rec_a, d_rec_b, c_x, c_rec_a, c_rec_b, training=False):
        loss_adv_dis = tf.reduce_mean(-d_x + d_rec_b, axis=0)
        loss_att_dis = criterion_BCE(a, c_x)

        if training is True:
            gp = utils.wgan_gp(x, x_rec_b, model.disc)
            loss_dis = loss_adv_dis + loss_att_dis + 10*gp
        else:
            loss_dis = loss_adv_dis + loss_att_dis

        # print("DISCRIMINATOR")
        # print("LOSS_DIS:", loss_dis)
        # print("ADV:", loss_adv_dis)
        # print("ATT:", loss_att_dis)
        # print("GP:", gp)
        # print()

        return loss_dis

In [10]:
with strategy.scope():
    def forward_prop(x, a, training=False):
        n = x.shape[0]
        num_att = a.shape[1]
        b = utils.generate_attribute(n, num_att)

        x_rec_a, c_rec_a, d_rec_a = model(x, a, training=training)
        x_rec_b, c_rec_b, d_rec_b = model(x, b, training=training)
        c_x, d_x = model.disc(x, training=training)

        # print(c_x)

        loss_gen = generator_loss(x, a, b, x_rec_a, x_rec_b, d_x, d_rec_a, d_rec_b, c_x, c_rec_a, c_rec_b, training=training)
        loss_dis = discriminator_loss(x, a, b, x_rec_a, x_rec_b, d_x, d_rec_a, d_rec_b, c_x, c_rec_a, c_rec_b, training=training)

        return x_rec_a, x_rec_b, loss_gen, loss_dis
    
    def inference(x, a, training):
        if training is True:
            with tf.GradientTape(persistent=True) as tape:
                x_rec_a, x_rec_b, loss_gen, loss_dis = forward_prop(x, a, True)
            
            grad_gen = tape.gradient(loss_gen, [*model.encoder.trainable_variables, *model.decoder.trainable_variables])
            optimizer.apply_gradients(zip(grad_gen, [*model.encoder.trainable_variables, *model.decoder.trainable_variables]))

            grad_dis = tape.gradient(loss_dis, model.disc.trainable_variables)
            optimizer.apply_gradients(zip(grad_dis, model.disc.trainable_variables))
            
        else:
            x_rec_a, x_rec_b, loss_gen, loss_dis = forward_prop(x, a, False)

        loss_mean_gen.update_state(loss_gen)
        loss_mean_dis.update_state(loss_dis)
            
        return x_rec_a, x_rec_b
    
    def concat_distributed_tensor(distributed_tensor):
        tensor_list = []
        
        for device in distributed_tensor.devices:
            tensor_list.append(distributed_tensor.get(device))
            
        concat_tensor = tf.concat(tensor_list, axis=0)
        return concat_tensor
    
    @tf.function
    def distributed_inference(x, a, training):
        x_rec_a, x_rec_b = strategy.experimental_run_v2(inference, args=(x, a, training))
        
        x = concat_distributed_tensor(x)
        x_rec_a = concat_distributed_tensor(x_rec_a)
        x_rec_b = concat_distributed_tensor(x_rec_b)
        
        return x, x_rec_a, x_rec_b

In [11]:
@tf.function
def get_mean_loss(loss_mean_obj):
    loss = loss_mean_obj.result()
    loss_mean_obj.reset_states()
    return loss

In [12]:
def detransform_image(image):
    image = image*256 + 128
    image = np.clip(image, 0, 255)
    image = image.astype(np.uint8)
    
    return image

def plot_images_with_index(images, indices):
    for i, index in enumerate(indices):
        image = images[index].numpy()
        image = detransform_image(image)
        
        plt.subplot(1, len(indices), i+1)
        plt.imshow(image)

def plot_images(x_origin, x_rec_a, x_rec_b):
    indices = np.random.randint(BATCH_SIZE, size=3)
    
    print("Origin:")
    plt.figure(figsize=(4, 3))
    plot_images_with_index(x_origin, indices)
    plt.show()
    
    print("Reconstructed with attribute a:")
    plt.figure(figsize=(12, 3))
    plot_images_with_index(x_rec_a, indices)
    plt.show()
    
    print("Reconstructed with attribute b:")
    plt.figure(figsize=(12, 3))
    plot_images_with_index(x_rec_b, indices)
    plt.show()

In [13]:
less_valid_loss = 10000000.0

In [ ]:
with strategy.scope():
    for e in range(EPOCHS):
        for x_batch, att_batch in train_dloader:
            x, x_rec_a, x_rec_b = distributed_inference(x_batch, att_batch, training=True)

        train_loss_gen = get_mean_loss(loss_mean_gen)
        train_loss_dis = get_mean_loss(loss_mean_dis)

        for x_batch, att_batch in valid_dloader:
            x, x_rec_a, x_rec_b = distributed_inference(x_batch, att_batch, training=False)
            
        valid_loss_gen = get_mean_loss(loss_mean_gen)
        valid_loss_dis = get_mean_loss(loss_mean_dis)

        print("===============================================================================")
        print(f"Epochs {e+1}/{EPOCHS}")
        print(f"Train generator loss: {train_loss_gen:.8f}")
        print(f"Train discriminator loss: {train_loss_dis:.8f}")
        print(f"Valid generator loss: {valid_loss_gen:.8f}")
        print(f"Valid discriminator loss: {valid_loss_dis:.8f}")
            
        print("=== TRAIN SET ===")
        plot_images(x, x_rec_a, x_rec_b)
        
        print("=== VALID SET ===")
        plot_images(x, x_rec_a, x_rec_b)

        with open(LOG_FILE, "a") as logfile:
            logfile.write(f"Epochs {e+1}/{EPOCHS}\n")
            logfile.write(f"Train generator loss: {train_loss_gen:.8f}\n")
            logfile.write(f"Train discriminator loss: {train_loss_dis:.8f}\n")
            logfile.write(f"Valid generator loss: {valid_loss_gen:.8f}\n")
            logfile.write(f"Valid discriminator loss: {valid_loss_dis:.8f}\n")

        if less_valid_loss > valid_loss_gen:
            less_valid_loss = valid_loss_gen
            utils.save_model_with_source(model, "ckpts/attgan", "models")

INFO:tensorflow:batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 36 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
INFO:tensorflow:batch_all_reduce: 32 all-reduces with algorithm = nccl, num_packs = 1, agg_small_grads_max_bytes = 0 and agg_small_grads_max_group = 10
